In [1]:
import os

import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import gc
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import Callback 
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

Using TensorFlow backend.


## Helper Functions

In [2]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [3]:
def build_model(transformer, max_len=512, hidden_dim=32, num_of_classes=3):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    
    out = Dense(num_of_classes, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

## TPU Configs

In [4]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [5]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 96
MODEL = 'bert-base-multilingual-cased'

## Create fast tokenizer

In [6]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

## Load text data into memory

In [7]:
AUGMENT_DATA = True

dbpedia_df = pd.read_csv("/kaggle/input/test-06366/dbpedia.csv", sep="|")
faketest_dbpedia_df = dbpedia_df.sample(4400, random_state=42) # Test set from organizers has ca. 4400 examples
dbpedia_df.dropna(subset=['question'], inplace=True)   

for i in range(5):
    dbpedia_df["type_{0}".format(i+1)].fillna(value="NoneType")
    dbpedia_df["type_{0}".format(i+1)] = dbpedia_df["type_{0}".format(i+1)].astype(str)


if AUGMENT_DATA:
    common_de_df = dbpedia_df.copy()
    common_de_df.question = dbpedia_df.question_de

    common_fr_df = dbpedia_df.copy()
    common_fr_df.question = dbpedia_df.question_fr

    common_es_df = dbpedia_df.copy()
    common_es_df.question = dbpedia_df.question_es

    common_ru_df = dbpedia_df.copy()
    common_ru_df.question = dbpedia_df.question_ru

    common_df = dbpedia_df.append(common_de_df).append(common_fr_df).append(common_es_df).append(common_ru_df)
    del common_de_df, common_fr_df, common_es_df, common_ru_df
    
    dbpedia_df = common_df
    
dbpedia_df.drop(['question_de', 'question_fr', 'question_es', 'question_ru'], axis=1, inplace=True)
dbpedia_df.dropna(subset=['question'], inplace=True)    
dbpedia_df = dbpedia_df[~dbpedia_df.id.isin(faketest_dbpedia_df.id.values)]


X_db_train, X_db_test, y1_db_train, y1_db_test, y2_db_train, y2_db_test, y3_db_train, y3_db_test, y4_db_train, y4_db_test, y5_db_train, y5_db_test = train_test_split(dbpedia_df[dbpedia_df.category == "resource"].question.astype(str), 
                                                                dbpedia_df[dbpedia_df.category == "resource"].type_1,
                                                                dbpedia_df[dbpedia_df.category == "resource"].type_2,
                                                                dbpedia_df[dbpedia_df.category == "resource"].type_3,
                                                                dbpedia_df[dbpedia_df.category == "resource"].type_4,
                                                                dbpedia_df[dbpedia_df.category == "resource"].type_5,
                                                                test_size=0.33, random_state=42)

## DBpedia Resource classifier

In [8]:
y_train = [y1_db_train, y2_db_train, y3_db_train, y4_db_train, y5_db_train]
y_test = [y1_db_test, y2_db_test, y3_db_test, y4_db_test, y5_db_test]

In [9]:
%%time
#models_list = list()
le_list = list()

for i in range(2):
    encoder_db = LabelEncoder()
    encoder_db.fit(dbpedia_df[dbpedia_df.category == "resource"]["type_{0}".format(i+1)])
    encoded_Y_db_train = encoder_db.transform(y_train[i])
    encoded_Y_db_test = encoder_db.transform(y_test[i])
    dummy_y_db_train = np_utils.to_categorical(encoded_Y_db_train) # convert integers to dummy variables (i.e. one hot encoded)
    
    le_list.append(encoder_db)    
    
    np.save('encoder_type{0}.npy'.format(i+1), encoder_db.classes_)
    
    x_db_train = regular_encode(X_db_train.values.astype(str), tokenizer, maxlen=MAX_LEN)
    x_db_test = regular_encode(X_db_test.values.astype(str), tokenizer, maxlen=MAX_LEN)
    
    train_dataset_db = (
        tf.data.Dataset
        .from_tensor_slices((x_db_train, dummy_y_db_train))
        .repeat()
        .shuffle(2048)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    test_dataset_db = (
        tf.data.Dataset
        .from_tensor_slices(x_db_test)
        .batch(BATCH_SIZE)
    )
    
    with strategy.scope():
        transformer_layer = TFAutoModel.from_pretrained(MODEL)
        model = build_model(transformer_layer, max_len=MAX_LEN, num_of_classes=dbpedia_df[dbpedia_df.category == "resource"]["type_{0}".format(i+1)].nunique())
        
    n_steps = x_db_train.shape[0] // BATCH_SIZE
    train_history = model.fit(
        train_dataset_db,
        steps_per_epoch=n_steps,
        epochs=EPOCHS #,
        #callbacks=[early_stopping]
    )
    
    
    y_db_test_pred = np.argmax(model.predict(test_dataset_db, verbose=1), axis=1)
    
    #models_list.append(model)
    
    f1 = f1_score(encoded_Y_db_test, y_db_test_pred, average='weighted')
    print('F1: {0}'.format(f1))
    print('!!!TEST set is also multilingual now!!!')
    
    model.save("model_{0}".format(i+1))
    del model
    
    gc.collect()


Train for 1481 steps
Epoch 1/2
1481/1481 [==============================] - 353s 239ms/step - loss: 1.1939 - accuracy: 0.6989
Epoch 2/2
730/730 [==============================] - 63s 87ms/step
F1: 0.8516377272892571
!!!TEST set is also multilingual now!!!
Train for 1481 steps
Epoch 1/2
1481/1481 [==============================] - 355s 240ms/step - loss: 1.6621 - accuracy: 0.5616
Epoch 2/2
730/730 [==============================] - 64s 87ms/step
F1: 0.8067114208405781
!!!TEST set is also multilingual now!!!
CPU times: user 15min 30s, sys: 2min 31s, total: 18min 1s
Wall time: 28min 14s
